# Inferring
In this lesson, you will infer sentiment and topics from product reviews and news articles.

## Setup

In [3]:
import requests
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
api_key = os.environ['GOOGLE_API_KEY']

# Set the API endpoint and API key.
endpoint = "https://generativelanguage.googleapis.com/v1beta"

def get_completion(prompt):
   # Prepare the request body.
   request_body = {
       "contents": [
           {
               "parts": [
                   {
                       "text": prompt
                   }
               ]
           }
       ]
   }

   # Make the request.
   response = requests.post(
       f"{endpoint}/models/gemini-pro:generateContent?key={api_key}",
       json=request_body,
   )

   # Parse the response.
   response_json = response.json()

   generated_content = response_json["candidates"][0]['content']['parts'][0]['text']

   # Return the generated content.
   return generated_content

## Product review text

## Sentiment (positive/negative)

In [9]:
LED_Light_Review = """
"Needed a reliable LED light for my vehicle, 
and this one exceeded my expectations. 
It boasts excellent brightness and energy efficiency, 
all at a reasonable price. Delivery was swift, 
and the packaging ensured the light arrived in perfect condition. 
Installation was a breeze, thanks to clear instructions provided. 
Despite a minor issue with the wiring upon arrival, the company's responsive customer support promptly resolved the issue and sent a replacement part. 
Overall, I'm thoroughly impressed with the quality of the LED light and the exceptional service provided by the company."
"""
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{LED_Light_Review}'''
"""
response = get_completion(prompt)
print(response)

Positive


## Identify types of emotions

In [10]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{LED_Light_Review}'''
"""
response = get_completion(prompt)
print(response)

- satisfaction
- excitement
- content
- gratitude
- appreciation


## Identify anger

In [11]:
prompt = f"""
Is the writer of the following review expressing anger?
The review is delimited with triple backticks.
Give your answer as either yes or no,
if it is no provide me the reason why?

Review text: '''This LED light is fantastic! It's so bright and energy-efficient. I highly recommend it.'''
"""
response = get_completion(prompt)
print(response)


no
The review is expressing positive feelings towards the LED light.


## Extract product and company name from customer reviews

In [12]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: '''{LED_Light_Review}'''
"""
response = get_completion(prompt)
print(response)

```json
{
  "Item": "LED light",
  "Brand": "unknown"
}
```


## Doing multiple tasks at once

In [13]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{LED_Light_Review}'''
"""
response = get_completion(prompt)
print(response)

```JSON
{
  "Sentiment": "positive",
  "Anger": false,
  "Item": "LED light",
  "Brand": "unknown"
}
```


## Inferring topics

In [14]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level of satisfaction with the department they work at. 
The results revealed that ISRO (Indian Space Research Organisation) emerged as the most popular department, 
boasting a satisfaction rating of 92%.

One ISRO employee, Priya Patel, 
shared her thoughts on the findings, expressing, 
"It's no surprise that ISRO is leading in job satisfaction. 
Working here is a dream come true, 
with endless opportunities for innovation and collaboration. 
I feel privileged to contribute to India's space exploration efforts alongside such talented colleagues."

The results were warmly received by ISRO's management team, 
with Director Rajesh Kumar stating, 
"We are delighted to see such positive feedback from our employees. 
At ISRO, we strive to foster a culture of excellence and teamwork, 
and it's gratifying to know that our efforts are reflected in the high satisfaction levels reported by our staff."

However, the survey also highlighted areas for improvement, 
with some employees expressing concerns about workload and work-life balance. 
ISRO has committed to addressing these issues and has initiated measures to enhance employee well-being and job satisfaction.n across all departments.
"""
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

- Employee satisfaction
- Survey results
- ISRO excellence
- Workload concerns
- Well-being initiatives


In [15]:
response.split(sep=',')

['- Employee satisfaction\n- Survey results\n- ISRO excellence\n- Workload concerns\n- Well-being initiatives']

## Make a news alert for certain topics

In [16]:
topic_list = [
    "ISRO", "local government", "engineering", 
    "employee satisfaction", "federal government"
]
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as list with 0 or 1 for each topic.\

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

* ISRO: 1
* local government: 0
* engineering: 0
* employee satisfaction: 1
* federal government: 0


In [17]:
topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response.split(sep='\n')}
if 'nasa' in topic_dict and topic_dict['nasa'] == 1:
    print("ALERT: New ISRO story!")
else:
    print("No NASA story detected or count not equal to 1.")


No NASA story detected or count not equal to 1.


## SUMMARY:

# Inferring

## Key Objectives:

1. **Product Reviews:** Analyze sentiment (positive/negative), detect emotions like anger, and extract product and company names from customer reviews.
2. **Multitasking:** Learn to handle multiple tasks concurrently.
3. **Topic Inference:** Gain insights into inferring topics, identifying five key topics.
4. **News Alerts:** Create alerts for specific topics to stay updated with relevant news.

